In [93]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [94]:
df = pd.read_csv('youtube_vids_1st_pull.csv', index_col=0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,41eHLnSTEsY,День сто шестьдесят второй. Беседа с Alexey A...,2022-08-04,0,74,0
1,fV4DOFnHoPE,День сто шестьдесят первый. Беседа с Alexey A...,2022-08-03,1423833,263214,3433
2,bn9cq1JqDtc,На фронтах IV мировой. Беседа с Андрей Пионтк...,2022-08-03,497961,63296,797
3,vSyZ0p-HcgM,День сто шестидесятый. Беседа с Alexey Aresto...,2022-08-02,1691733,284706,3282
4,Dh9Dc6Sw3kY,День сто пятьдесят девятый. Беседа с Alexey A...,2022-08-01,1883506,302672,3847


In [95]:
def connect_to_db(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host = host_name,database = dbname,user = username,password = password,port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print("Connected!")
        return conn

In [96]:
#create table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                            video_id VARCHAR(255) PRIMARY KEY,
                            video_title TEXT NOT NULL,
                            upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                            view_count INTEGER NOT NULL,
                            like_count INTEGER NOT NULL,
                            comment_count INTEGER NOT NULL) """)
    curr.execute(create_table_command)

In [97]:
#write insert command
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, comment_count)
        VALUES (%s, %s, %s, %s, %s, %s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [98]:
#update row if video exists
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
                SET video_title = %s,
                    view_count = %s,
                    like_count = %s,
                    comment_count = %s
                WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [99]:
#check to see if video exists
def check_if_video_exists(curr,video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query,(video_id,))
    
    return curr.fetchone() is not None


In [100]:
def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)

In [101]:
def append_from_df_to_db(curr,df):
    for i, rows in df.iterrows():
        insert_into_table(curr, rows['video_id'], rows['video_title'], rows['upload_date'], rows['view_count'], rows['like_count'], rows['comment_count'])

In [102]:
#updating the database
#handling data for scalability

#check to see if video exists
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title','upload_date', 'view_count', 'like_count', 'comment_count'])

    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): #if video already exiscts we will update
            update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count'])
        else: # the video does not exist so we need to append it to the df
            tmp_df = tmp_df.append(row)
    return tmp_df     
#perform update on existing video

#perform an insert on new videos

In [103]:
#Main

#database credentials
host_name = 'xxxxxxxxxxxxxx.rds.amazonaws.com'
dbname = 'myDatabase' 
port = '5432'
username = 'xxxxxxxxxxx'
password = 'xxxxxxxxxxx'
conn = None

#establish a connection to db
conn = connect_to_db(host_name,dbname, username, password, port)
curr = conn.cursor()

Connected!


In [85]:
create_table(curr)

In [112]:
#update data for existing videos

new_vid_df = update_db(curr, df)
new_vid_df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,41eHLnSTEsY,День сто шестьдесят второй. Беседа с Alexey A...,2022-08-04,0,74,0
1,fV4DOFnHoPE,День сто шестьдесят первый. Беседа с Alexey A...,2022-08-03,1423833,263214,3433
2,bn9cq1JqDtc,На фронтах IV мировой. Беседа с Андрей Пионтк...,2022-08-03,497961,63296,797
3,vSyZ0p-HcgM,День сто шестидесятый. Беседа с Alexey Aresto...,2022-08-02,1691733,284706,3282
54,lvpI9FxeBDE,Кооператив &quot;Озеро&quot; и термоядерная во...,2022-06-28,775962,68302,1292
...,...,...,...,...,...,...
306,xr0RwxyvrVQ,Сурков и &quot;украинский вопрос&quot;. Беседа...,2020-02-26,51419,4880,167
307,V3FEL6QmNeM,Подальше от Путина. Беседа с Аркадием Бабченко,2020-01-28,89196,7666,503
308,X5xVR_U9K8A,Путин и Питер. Беседа с Дмитрием Запольским,2019-11-26,228399,10796,604
309,fzkMnFdCoiQ,Русская эмиграция и русская культура,2019-10-29,9123,875,80


In [111]:
conn.commit()

In [113]:
new_vid_df[new_vid_df['video_id']=='so7sPx9Dhpk']

,video_id,video_title,upload_date,view_count,like_count,comment_count
99,so7sPx9Dhpk,Бои в городах. Беседа с Олег Жданов,2022-05-23,873299,88988,1190
100,so7sPx9Dhpk,Бои в городах. Беседа с Олег Жданов,2022-05-23,873299,88988,1190


In [114]:
new_vid_df.drop(new_vid_df[new_vid_df['video_id'] =='so7sPx9Dhpk'].index, inplace=True)

In [115]:
new_vid_df[new_vid_df['video_id']=='so7sPx9Dhpk']

,video_id,video_title,upload_date,view_count,like_count,comment_count


In [116]:
#insert new videos into db table
append_from_df_to_db(curr,new_vid_df)
conn.commit()

In [117]:
#view data in db table

curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('Dh9Dc6Sw3kY', 'День сто пятьдесят девятый. Беседа с  Alexey Arestovych Алексей Арестович', datetime.date(2022, 8, 1), 1883506, 302672, 3847, None), ('IGHXVvpLbEc', 'Две угрозы. Беседа с Юрием Фёдоровым', datetime.date(2022, 7, 31), 933966, 88567, 850, None), ('1n0r23kPCSg', 'Концлагерь в Еленовке. Беседа с Антоном Геращенко', datetime.date(2022, 7, 30), 1039028, 117737, 3655, None), ('cArmrPDp8sY', 'День сто пятьдесят шестой. Беседа с  Alexey Arestovych Алексей Арестович', datetime.date(2022, 7, 29), 2049854, 299525, 5209, None), ('3hj0QtW0WJw', 'Война до последнего русского. Беседа с Фёдором Крашенинниковым', datetime.date(2022, 7, 28), 837709, 73468, 2295, None), ('RKHAHi8K3IA', 'День сто пятьдесят четвёртый. Беседа с  Alexey Arestovych Алексей Арестович', datetime.date(2022, 7, 27), 1787524, 287279, 4055, None), ('7FrhwpCb570', 'Правый берег. Беседа с  Олег Жданов', datetime.date(2022, 7, 27), 916989, 111604, 1248, None), ('9fuz_mz6PUk', 'Перемирие. Беседа с  Андрей Пионтковский'